In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans

In [5]:
df = pd.read_pickle('data/latest_merged.pkl')

In [6]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)
pd.set_option("display.max_colwidth",None)

In [7]:
df.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,No_of_PreviousApplications,Credit_to_Income_Ratio,Annuity_to_Income_Ratio,GoodsPrice_to_Income_Ratio,No_of_DocumentsSubmitted,EXT_SCORES_AVG,EXT_SCORES_SUM,FLAG_CONTACT_INFO,EXT_SOURCE_1,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Security staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_XNA,clusters,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,HOUR_APPR_PROCESS_START_y_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,DAYS_DECISION_list,SELLERPLACE_AREA_list,CNT_PAYMENT_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list,NAME_CONTRACT_TYPE_y_list_Cash loans,NAME_CONTRACT_TYPE_y_list_Consumer loans,NAME_CONTRACT_TYPE_y_list_Revolving loans,NAME_CONTRACT_TYPE_y_list_XNA,WEEKDAY_APPR_PROCESS_START_y_list_FRIDAY,WEEKDAY_APPR_PROCESS_START_y_list_MONDAY,WEEKDAY_APPR_PROCESS_START_y_list_SATURDAY,WEEKDAY_APPR_PROCESS

In [8]:
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from xgboost import XGBClassifier

In [ ]:
cat = df.select_dtypes(include='object').columns
lb = LabelEncoder()
for col in cat:
  df[col] = lb.fit_transform(df[col])

In [9]:
def split(dataframe):
    X = dataframe.drop('TARGET',axis=1)
    y = dataframe['TARGET']
    y = y.astype('int')

    X_train1,X_test,y_train1,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

    X_train,X_val,y_train,y_val = train_test_split(X_train1,y_train1,test_size=0.2,random_state=0)
    return X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val

def fit_predict(model):
    model.fit(X_train,y_train)

    y_pred = model.predict(X_val)

    print("Accuracy = {0} \nRecall {1} \nPrecision {2} \nF1 score {3}".format( 
        accuracy_score(y_val,y_pred),recall_score(y_val,y_pred),
        precision_score(y_val,y_pred),f1_score(y_val,y_pred)))

def cross_validate(model,scoring_metric,folds):  
    skf = StratifiedKFold(folds,shuffle=True,random_state=0)
    scores = cross_val_score(log,X_train1,y_train1,cv=skf,scoring=scoring_metric)

    print(scores)
    print(np.mean(scores))

def feature_importance(model):

    feature_imp =  pd.DataFrame(model.feature_importances_,index=list(df.columns.drop('TARGET')),
              columns=['feature_imp']).sort_values('feature_imp',ascending=False)
    imp_columns = list(feature_imp[feature_imp['feature_imp']>0].index)
    imp_df = df[imp_columns]
    imp_df['TARGET'] = df['TARGET']
    return feature_imp,imp_df

In [10]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(df)

In [ ]:
log = LogisticRegression(random_state=42)
fit_predict(log)

Accuracy = 0.9189062229584395 
Recall 0.014889091461561836 
Precision 0.5632183908045977 
F1 score 0.02901124925991711


In [11]:
log.fit(X_train,y_train)
y_pred = log.predict(X_val)

In [15]:
pd.Series(y_pred)

0        0
1        0
2        0
3        0
4        0
        ..
40442    0
40443    0
40444    0
40445    0
40446    0
Length: 40447, dtype: int64

In [16]:
y_val.to_csv("y_actual.csv")
pd.Series(y_pred).to_csv("y_pred.csv")

In [ ]:
scoring_metric='accuracy'
cross_validate(log,scoring_metric,20)

[0.9180182  0.91762263 0.91772152 0.91861155 0.9180182  0.91772152
 0.91782041 0.9180182  0.91851266 0.91871044 0.9166337  0.91831487
 0.91732595 0.9179193  0.91810899 0.91771338 0.91810899 0.91721887
 0.91771338 0.91791119]
0.9178871972646346


In [ ]:
coeff_weights = pd.read_excel('/content/drive/MyDrive/CoeffWeights-Logit_20210831.xlsx')
coeff_weights

,Rank Ordered,Features,Coefficient,Abs Coeff,Normalized,Weights (in %),Cummulative Weight (in %)
0,1.0,AMT_GOODS_PRICE,-0.886893,0.886893,0.035191,3.519112,3.519112
1,2.0,AMT_CREDIT,0.840607,0.840607,0.033355,3.335451,6.854564
2,3.0,CHANNEL_TYPE_list_Channel of corporate sales,-0.640278,0.640278,0.025406,2.540564,9.395128
3,4.0,FLAG_DOCUMENT_18,-0.489143,0.489143,0.019409,1.940874,11.336001
4,5.0,FLAG_DOCUMENT_11,-0.418793,0.418793,0.016617,1.661733,12.997734
...,...,...,...,...,...,...,...
224,225.0,NAME_PORTFOLIO_list_POS,0.001788,0.001788,0.000071,0.007095,99.994124
225,226.0,WEEKDAY_APPR_PROCESS_START_FRIDAY,0.000951,0.000951,0.000038,0.003772,99.997896
226,227.0,OCCUPATION_TYPE_Sales staff,-0.000530,0.000530,0.000021,0.002104,100.000000
227,228.0,FLAG_DOCUMENT_12,0.000000,0.000000,0.000000,0.000000,100.000000


In [ ]:
coeff_columns =  list(coeff_weights[:42]['Features'])
coeff_columns_df = df[coeff_columns]
coeff_columns_df['TARGET'] = df['TARGET']

In [ ]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(coeff_columns_df)
log = LogisticRegression(random_state=42)
fit_predict(log)

Accuracy = 0.9187578806833634 
Recall 0.010027347310847767 
Precision 0.5409836065573771 
F1 score 0.019689737470167064


In [ ]:
log = LogisticRegression(random_state=42)
cross_validate(log,'accuracy',20)

[0.91841377 0.9179193  0.91821598 0.91752373 0.91772152 0.9179193
 0.91811709 0.91871044 0.91811709 0.91851266 0.91752373 0.91851266
 0.91762263 0.91772152 0.91820789 0.91761448 0.91781228 0.91791119
 0.91801009 0.9186035 ]
0.9180355427187022


In [ ]:
dt = DecisionTreeClassifier(random_state=0)

In [ ]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(df)

In [ ]:
fit_predict(dt)

Accuracy = 0.8494573145103469 
Recall 0.16560316013369797 
Precision 0.14017489711934156 
F1 score 0.15183173143891907


In [ ]:
cross_validate(dt,'accuracy',20)

In [ ]:
rf = RandomForestClassifier(random_state=0)

fit_predict(rf)

Accuracy = 0.9187084332583381 
Recall 0.0018231540565177757 
Precision 0.6666666666666666 
F1 score 0.0036363636363636364


In [ ]:
cross_validate(rf,'accuracy',20)

In [ ]:
ada = AdaBoostClassifier(random_state=0)

fit_predict(ada)

Accuracy = 0.9189062229584395 
Recall 0.028866605894864783 
Precision 0.5307262569832403 
F1 score 0.05475504322766571


In [ ]:
cross_validate(ada,'accuracy',20)

In [ ]:
ada_feature_imp =  pd.DataFrame(ada.feature_importances_,index=list(df.columns.drop('TARGET')),
             columns=['feature_imp']).sort_values('feature_imp',ascending=False)
ada_feature_imp['feature_imp'].sum()

1.0

In [ ]:
gb = GradientBoostingClassifier(random_state=0)

fit_predict(gb)

Accuracy = 0.9193018023586422 
Recall 0.017927681555758128 
Precision 0.6483516483516484 
F1 score 0.03489059727971615


In [ ]:
cross_validate(gb,'accuracy',20)

[0.91811709 0.91772152 0.91782041 0.91880934 0.9180182  0.91772152
 0.91782041 0.9179193  0.91851266 0.91861155 0.91673259 0.91851266
 0.91722706 0.9180182  0.91830679 0.91771338 0.91820789 0.91702107
 0.91761448 0.91791119]
0.9179168649861538


In [ ]:
gb_feature_imp,gb_imp_df =  feature_importance(gb)

In [ ]:
imp_cols = list(gb_feature_imp[:8].index)
gb_imp_df = df[imp_cols]
gb_imp_df['TARGET'] = df['TARGET']

In [ ]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(gb_imp_df)

In [ ]:
fit_predict(gb)

Accuracy = 0.9187331569708508 
Recall 0.011546642357945914 
Precision 0.5277777777777778 
F1 score 0.02259887005649718


In [ ]:
gb_feature_imp[gb_feature_imp['feature_imp']>0].to_excel('gb_feature_imp.xlsx')

In [ ]:
xgb = XGBClassifier(random_state=0,n_jobs=-1)


In [ ]:
fit_predict(xgb)

Accuracy = 0.9188814992459268 
Recall 0.005165603160133698 
Precision 0.7083333333333334 
F1 score 0.010256410256410256


In [ ]:
cross_validate(xgb,'accuracy',20)

[0.91811709 0.91772152 0.91782041 0.91880934 0.9180182  0.91772152
 0.91782041 0.9179193  0.91851266 0.91861155 0.91673259 0.91851266
 0.91722706 0.9180182  0.91830679 0.91771338 0.91820789 0.91702107
 0.91761448 0.91791119]
0.9179168649861538


In [ ]:
xgb_feature_imp =  pd.DataFrame(xgb.feature_importances_,index=list(df.columns.drop('TARGET')),
             columns=['feature_imp']).sort_values('feature_imp',ascending=False)
xgb_feature_imp['feature_imp'].sum()

1.0

In [ ]:
imp_columns = list(xgb_feature_imp[xgb_feature_imp['feature_imp']>0].index)
xgb_imp_df = df[imp_columns]
xgb_imp_df['TARGET'] = df['TARGET']

In [ ]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(xgb_imp_df)

In [ ]:
xgb = XGBClassifier(random_state=0)

fit_predict(xgb)

Accuracy = 0.9188814992459268 
Recall 0.005165603160133698 
Precision 0.7083333333333334 
F1 score 0.010256410256410256


### Hyperparameter Tuning

In [ ]:
gb = GradientBoostingClassifier(random_state=0)

learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0

In [ ]:
params = {
    'n_estimators':[100,150],
    'max_depth':[3,4]
}

In [ ]:
random_cv = RandomizedSearchCV(gb,param_distributions=params,n_jobs=-1,verbose=2,random_state=0)

In [ ]:
random_cv.fit(X_train1,y_train1)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

| Model               	| Scores  	|
|---------------------	|---------	|
| Logistic Regression 	| 91.8906 	|
| Decision Tree       	| 84.9457 	|
| Random Forest       	| 91.8708 	|
| AdaBoost            	| 91.8906 	|
| Gradient Boost      	| 91.9301 	|
| XGBoost             	| 91.8881 	|